# Randall Reese
## Take Home Exercise: Recursion, Data Science Generalist application. March 2, 2021

See "Recursion Notes.pdf" for further notes.

-----------

Setting up the basic imports.

In [258]:
import numpy as np
import pandas as pd

Setup the specialized imports.

In [259]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


import tensorflow as tf ##May not eeven use TF. 


-----------

In [260]:
metadata = pd.read_csv("RxRx19a-Meta/metadata.csv", usecols = [0,1,2,3,4,5,6,7,8,9])

In [261]:
metadata.head(20)

,site_id,well_id,cell_type,experiment,plate,well,site,disease_condition,treatment,treatment_conc
0,HRCE-1_1_AA02_1,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,1,Active SARS-CoV-2,Flubendazole,0.10
1,HRCE-1_1_AA02_2,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,2,Active SARS-CoV-2,Flubendazole,0.10
2,HRCE-1_1_AA02_3,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,3,Active SARS-CoV-2,Flubendazole,0.10
3,HRCE-1_1_AA02_4,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,4,Active SARS-CoV-2,Flubendazole,0.10
4,HRCE-1_1_AA03_1,HRCE-1_1_AA03,HRCE,HRCE-1,1,AA03,1,Active SARS-CoV-2,acetylcysteine,1.00
5,HRCE-1_1_AA03_2,HRCE-1_1_AA03,HRCE,HRCE-1,1,AA03,2,Active SARS-CoV-2,acetylcysteine,1.00
6,HRCE-1_1_AA03_3,HRCE-1_1_AA03,HRCE,HRCE-1,1,AA03,3,Active SARS-CoV-2,acetylcysteine,1.00
7,HRCE-1_1_AA03_4,HRCE-1_1_AA03,HRCE,HRCE-1,1,AA03,4,Active SARS-CoV-2,acetylcysteine,1.00
8,HRCE-1_1_AA04_1,HRCE-1_1_AA04,HRCE,HRCE-1,1,AA04,1,Active SARS-CoV-2,Clopidol,0.01
9,HRCE-1_1_AA04_2,HRCE-1_1_AA04,HRCE,HRCE-1,1,AA04,2,Active SARS-CoV-2,Clopidol,0.01


In [262]:
metadata['disease_condition'].value_counts() ##How many of each condition are there?

Active SARS-CoV-2            280376
UV Inactivated SARS-CoV-2      9120
Mock                           9120
Name: disease_condition, dtype: int64

In [263]:
treat_freq = metadata['treatment'].value_counts() ##Lots of treatment types. This is a dict.

In [264]:
metadata['cell_type'].value_counts()

HRCE    284080
VERO     21440
Name: cell_type, dtype: int64

A basic readout in Pandas of the data shows some basic dimensions and what the data looks like. SMILES seems to be extraneous. Just for simplicity and cleanliness, I'm going to completely ignore it for this exercise. 


The disease_cond, treatment and treat_conc categories of the meta data have missing values. Are these the mocks? It would seem like it would be pretty easy to verify this with the data creator if I actually was doing this in real life. I am going to treat them as unused wells and drop them. This likely afftecs the image to have nothing in the well, but it complicates things beyond what I could consider in a speedy analysis. There are some rows also with just "mock" and then no treatement. Impute those as (mock, none, 0). 

When I have Active SARS-CoV-2 and the treatment entries are blank, I'm treating those as (Active Covid, None, 0). So, no treatment done at those sites.

There also is the UV inactive sites. Those also seem to have no treatment. Impute as (UV, None, 0).

In [265]:
metadata.loc[metadata['disease_condition'].isna()]##Empty wells (?). Sounds plausible. 

,site_id,well_id,cell_type,experiment,plate,well,site,disease_condition,treatment,treatment_conc
392,HRCE-1_1_AC08_1,HRCE-1_1_AC08,HRCE,HRCE-1,1,AC08,1,NaN,NaN,NaN
393,HRCE-1_1_AC08_2,HRCE-1_1_AC08,HRCE,HRCE-1,1,AC08,2,NaN,NaN,NaN
394,HRCE-1_1_AC08_3,HRCE-1_1_AC08,HRCE,HRCE-1,1,AC08,3,NaN,NaN,NaN
395,HRCE-1_1_AC08_4,HRCE-1_1_AC08,HRCE,HRCE-1,1,AC08,4,NaN,NaN,NaN
424,HRCE-1_1_AC16_1,HRCE-1_1_AC16,HRCE,HRCE-1,1,AC16,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
301571,VERO-2_2_D32_4,VERO-2_2_D32,VERO,VERO-2,2,D32,4,NaN,NaN,NaN
301600,VERO-2_2_D40_1,VERO-2_2_D40,VERO,VERO-2,2,D40,1,NaN,NaN,NaN
301601,VERO-2_2_D40_2,VERO-2_2_D40,VERO,VERO-2,2,D40,2,NaN,NaN,NaN
301602,VERO-2_2_D40_3,VERO-2_2_D40,VERO,VERO-2,2,D40,3,NaN,NaN,NaN


In [266]:
nonemptyWells = metadata[~metadata['disease_condition'].isnull()].index.tolist()

metadata = metadata.iloc[nonemptyWells] ## Remove empty wells. 

Impute the missing data as discussed above. Since this meta data was (seemingly) computer generated (I assume this was not hand entered), I'm not sure why a place holder value was not used in the original creation of the data set.

In [267]:
metadata['treatment'] = metadata['treatment'].fillna("none")
metadata['treatment_conc'] = metadata['treatment_conc'].fillna(0)

In [268]:
metadata.loc[metadata['treatment'] == "none"] ##Check that the imputations have worked. 

,site_id,well_id,cell_type,experiment,plate,well,site,disease_condition,treatment,treatment_conc
24,HRCE-1_1_AA08_1,HRCE-1_1_AA08,HRCE,HRCE-1,1,AA08,1,UV Inactivated SARS-CoV-2,none,0.0
25,HRCE-1_1_AA08_2,HRCE-1_1_AA08,HRCE,HRCE-1,1,AA08,2,UV Inactivated SARS-CoV-2,none,0.0
26,HRCE-1_1_AA08_3,HRCE-1_1_AA08,HRCE,HRCE-1,1,AA08,3,UV Inactivated SARS-CoV-2,none,0.0
27,HRCE-1_1_AA08_4,HRCE-1_1_AA08,HRCE,HRCE-1,1,AA08,4,UV Inactivated SARS-CoV-2,none,0.0
56,HRCE-1_1_AA16_1,HRCE-1_1_AA16,HRCE,HRCE-1,1,AA16,1,UV Inactivated SARS-CoV-2,none,0.0
...,...,...,...,...,...,...,...,...,...,...
305503,VERO-2_2_Z43_4,VERO-2_2_Z43,VERO,VERO-2,2,Z43,4,Active SARS-CoV-2,none,0.0
305516,VERO-2_2_Z47_1,VERO-2_2_Z47,VERO,VERO-2,2,Z47,1,Active SARS-CoV-2,none,0.0
305517,VERO-2_2_Z47_2,VERO-2_2_Z47,VERO,VERO-2,2,Z47,2,Active SARS-CoV-2,none,0.0
305518,VERO-2_2_Z47_3,VERO-2_2_Z47,VERO,VERO-2,2,Z47,3,Active SARS-CoV-2,none,0.0


In [269]:
##disease_code =  {"Active SARS-CoV-2":0, "UV Inactivated SARS-CoV-2":1, "Mock":2}

## I am going to encode the disease conditions as 0,1,2. 
## This is mostly just to avoid having to work with strings as targets. Numbers are much cleaner. 

conditions = [
    (metadata['disease_condition'] =="Active SARS-CoV-2"),
    (metadata['disease_condition'] =="UV Inactivated SARS-CoV-2"),
    (metadata['disease_condition'] =="Mock") 
    ]

metadata['dis_code'] = np.select(conditions, [0,1,2])

print(metadata['dis_code'].value_counts()) ##Make sure that the encodings worked

print("\n")

print(metadata['disease_condition'].value_counts())

## Adding also an encoding for diseased or non-diseased. This is used later for the scoring. 
disease_state = [
    (metadata['disease_condition'] =="Active SARS-CoV-2") & (metadata['treatment'] == "none"),
    (metadata['disease_condition'] !="Active SARS-CoV-2"),     
    ]

metadata['dis_state'] = np.select(disease_state, [-1,1]) ## -1 is untreated diseased. +1 is UV/Mock.  
## By default, the remaining cells (unknown state, Active COVID) are given the value of 0. 

print("\n\n",metadata['dis_state'].value_counts()) ##Make sure that the encodings worked


0    280376
1      9120
2      9120
Name: dis_code, dtype: int64


Active SARS-CoV-2            280376
UV Inactivated SARS-CoV-2      9120
Mock                           9120
Name: disease_condition, dtype: int64


  0    263184
 1     18240
-1     17192
Name: dis_state, dtype: int64


In [270]:
metadata.to_csv(path_or_buf = "RxRx19a-Meta/cleanedMeta.csv")##Save this cleaned data for backup. 
metadata = metadata.reset_index(drop = True)##Otherwise we get indexing probalems later on. 
metadata

,site_id,well_id,cell_type,experiment,plate,well,site,disease_condition,treatment,treatment_conc,dis_code,dis_state
0,HRCE-1_1_AA02_1,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,1,Active SARS-CoV-2,Flubendazole,0.1,0,0
1,HRCE-1_1_AA02_2,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,2,Active SARS-CoV-2,Flubendazole,0.1,0,0
2,HRCE-1_1_AA02_3,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,3,Active SARS-CoV-2,Flubendazole,0.1,0,0
3,HRCE-1_1_AA02_4,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,4,Active SARS-CoV-2,Flubendazole,0.1,0,0
4,HRCE-1_1_AA03_1,HRCE-1_1_AA03,HRCE,HRCE-1,1,AA03,1,Active SARS-CoV-2,acetylcysteine,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
298611,VERO-2_2_Z46_4,VERO-2_2_Z46,VERO,VERO-2,2,Z46,4,Active SARS-CoV-2,Lopinavir,1.0,0,0
298612,VERO-2_2_Z47_1,VERO-2_2_Z47,VERO,VERO-2,2,Z47,1,Active SARS-CoV-2,none,0.0,0,-1
298613,VERO-2_2_Z47_2,VERO-2_2_Z47,VERO,VERO-2,2,Z47,2,Active SARS-CoV-2,none,0.0,0,-1
298614,VERO-2_2_Z47_3,VERO-2_2_Z47,VERO,VERO-2,2,Z47,3,Active SARS-CoV-2,none,0.0,0,-1


In [271]:
##embeddings = pd.read_csv("RxRx19a/embeddings.csv", index_col = 0, skiprows=lambda w: w not in nonemptyWells)
##A little awkward, but read_csv only lets you exclude rows.
## And it might be faster to read the whole data set in and then index (?)
embeddings = pd.read_csv("RxRx19a/embeddings.csv", index_col = 0)
embeddings = embeddings.iloc[nonemptyWells]
##embeddings = embeddings.reset_index(drop = True)##Otherwise we get indexing probalems later on. 

In [272]:
np.shape(embeddings) ##Shape looks good. 

(298616, 1024)

In [273]:
np.shape(metadata) ## 298,616 records. 

(298616, 12)

In [274]:
embeddings.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_1014,feature_1015,feature_1016,feature_1017,feature_1018,feature_1019,feature_1020,feature_1021,feature_1022,feature_1023
site_id,,,,,,,,,,,,,,,,,,,,,
HRCE-1_10_AA02_1,2.355969,-0.058361,-0.169764,-0.316499,-0.891334,0.581174,-0.284587,-0.279198,-0.146575,-0.175336,...,0.391186,0.786199,-1.739626,-1.317543,-1.208275,-0.507439,-0.317298,0.285018,-0.091285,-1.553895
HRCE-1_10_AA02_2,2.325652,-0.202519,-0.296017,-0.481136,-0.641461,0.702847,0.334191,-0.077498,-0.314538,-0.589848,...,0.638674,1.051621,-1.355659,-1.285210,-1.341911,-0.271349,-0.157707,0.081128,-0.447174,-1.614872
HRCE-1_10_AA02_3,2.207082,-0.379794,-0.365562,-0.196667,-0.799039,0.735813,0.081227,-0.393452,0.066324,-0.310497,...,0.552127,0.775428,-1.616731,-0.868382,-1.334486,-0.774456,-0.363250,0.161023,-0.066745,-1.325545
HRCE-1_10_AA02_4,2.452741,0.050658,-0.444642,-0.309758,-0.724900,0.658327,-0.128331,-0.164759,0.279105,-0.344210,...,0.689145,1.167066,-1.275786,-2.088455,-1.231964,-0.364454,-0.135066,0.493010,-0.544680,-1.290150
HRCE-1_10_AA03_1,2.106868,0.032095,-0.175542,-0.808618,-0.806217,0.721340,-0.213196,-0.187649,-0.070294,0.077702,...,0.385696,0.503924,-1.192554,-1.123965,-1.323457,-0.359761,-0.100340,0.185773,-0.394267,-1.712760


The data seems to be in order initially. 
The aim right now is a minimal viable product, not a full scale workup. Something that could be done in 2 or so hours.

Based on the computing and time resources I have, a simpler model is likely going to be more feasible. Ideally I'd like to pursue a neural network. 1024 features is quite a few for simpler models (although I'd guess that 1024 input neurons to a neural network would work quite well). I'm hesitant to try to get a well formed and working neural network off the ground in a rather short amount of time, especially where I don't have personal-use access to a HPC platform and am just using a thin laptop. 

I am going to reduce the feature space further using principal components. Ideally I would know a bit more about how our current embedding of 1024 features is created. Where is this embedding coming from? Is it PCA-based itself? I would want to know more about this in a "real life" setting. But here, given our constraints, it's likely not feasible to have 1024 features in the model and train everything quickly. When you are working on a minimal viable solution, you have to reduce the data space and make some educated guesses. I just need to keep things somewhat simple and see what we can get. 

However, if we can get decent explanation of variance using just a few principal components, going forward with just these few features might lead us to a simpler viable solution. I also partially base this off of the concept of model sparsity which postulates that most features are superfluous.

------------------------


In [275]:
#pca = PCA(n_components=30 )
pca = PCA(0.9) ##90% Variance explained requires 63 PCs. Is that too much? Too little? with rf probably okay. 
##Choose a component number that gives enough explanation of variance.

In [276]:
embeddings_stdz = StandardScaler().fit_transform(embeddings) ##Standardize the embeddings data. 

In [277]:
embeddings_pc = pca.fit_transform(embeddings_stdz)

In [278]:
embeddings_pc = pd.DataFrame(data = embeddings_pc) 
embeddings_pc.columns=["v_" + str(idx) for idx in range(pca.n_components_)]##Clean up the column names a bit. 

In [279]:
embeddings_pc

,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,...,v_53,v_54,v_55,v_56,v_57,v_58,v_59,v_60,v_61,v_62
0,-5.648822,-9.701595,-4.389027,-3.724540,1.750453,-10.218108,6.698899,-8.002185,2.194457,0.525182,...,1.138334,0.623012,-0.502271,-2.143368,0.012130,-0.953206,1.009449,0.106435,-0.107482,-0.371420
1,-11.236373,-11.383790,-1.669045,-0.880511,-0.002458,-5.740356,6.724205,-5.036800,1.103920,3.161690,...,-2.014080,3.645765,0.137242,-0.486441,-1.513938,1.475253,-0.765367,0.615620,-2.178531,-1.465243
2,-7.570543,-8.762654,2.372159,-3.400681,1.610322,-7.787719,10.060818,-5.434962,-1.070973,2.536703,...,1.119752,0.249184,-0.414499,1.632017,-0.261208,-0.466153,2.193993,-0.877389,0.695840,-1.927933
3,-5.493966,-8.980262,-1.576731,-5.232435,1.818481,-5.916596,2.929959,-8.541632,3.362297,-3.085474,...,0.860601,2.240357,-0.868935,-1.006250,1.326486,-0.609907,-2.981634,1.084425,-0.297733,-2.423678
4,-3.596529,-11.059896,-5.329073,4.466943,6.747674,3.110455,10.896265,-4.314780,-3.210623,6.774098,...,1.607521,-1.724079,-0.893511,-2.345679,0.180880,0.523543,-0.288031,0.841113,0.975545,0.806070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298611,56.808579,0.797516,-11.088070,0.906256,-5.890064,-1.767247,4.967035,-3.297416,13.472089,3.067630,...,-1.306542,-1.074133,2.107548,-1.093986,0.103355,0.958140,2.030790,-0.703127,1.419976,-0.314094
298612,53.929497,1.082175,-10.786287,-0.599606,-6.816575,2.214472,1.177641,-0.456160,12.902565,3.133180,...,2.660205,-0.099500,0.644587,-0.884825,3.294007,-1.745628,-2.120144,0.214709,1.374555,-2.580990
298613,51.549044,-2.033512,-13.926395,-2.092892,-7.773888,-3.409473,2.610050,-14.382703,14.449253,1.077918,...,0.779405,1.038199,1.121651,2.601514,-2.336122,0.558548,0.171151,-1.581038,0.545468,-1.450148
298614,55.396253,0.705387,-6.647546,-1.199119,-9.355433,0.447931,2.327198,0.463805,7.954021,3.746620,...,2.340552,-0.846141,2.917409,1.619582,0.586522,-0.211522,-0.316186,-1.169499,0.400665,0.247488


In [280]:
pca.explained_variance_ratio_

array([0.2427855 , 0.13408162, 0.05463073, 0.05253271, 0.03824956,
       0.02933086, 0.02635562, 0.02328414, 0.02010943, 0.01818587,
       0.01497408, 0.01438109, 0.01360472, 0.01296386, 0.01140674,
       0.01023021, 0.00949089, 0.00902235, 0.00819155, 0.0073269 ,
       0.00716276, 0.00663136, 0.00620805, 0.00592315, 0.00548758,
       0.00526364, 0.00511655, 0.00489054, 0.00460977, 0.00451751,
       0.00443012, 0.0042025 , 0.00411361, 0.00386636, 0.00372382,
       0.00365921, 0.00363436, 0.00350953, 0.00340319, 0.0032513 ,
       0.0032102 , 0.00309292, 0.00298903, 0.00296599, 0.00290655,
       0.00273523, 0.00263809, 0.0025748 , 0.00252901, 0.00249706,
       0.00239279, 0.00235677, 0.00230659, 0.00218384, 0.0021621 ,
       0.00212304, 0.00208773, 0.00204293, 0.0020163 , 0.00197153,
       0.00192952, 0.00189672, 0.00188032])

In [281]:
np.sum(pca.explained_variance_ratio_)##The target was to explain 90% of the variance with the PCA. 
## 90% really is just a very rough rule of thumb. 
## As long as I could reduce the feature space somewhat and get a good amount of variance explained, I was happy.

0.9002323935962893

I am going to be fitting a Random Forest (RF) model. RF can handle more predictors than 63 (even several 100 predictors is okay), but sometimes it takes much longer to fit the model. With the aim at being somewhat fast here, I'm just using PCA to get a smaller feature set in hopes of some faster training. But, in a more extended setting, we might actually be okay using the entire embedding feature space.

I am going to fit three RF models. One model just to the HRCE data; one to the VERO data; and one model using both cell types. I will need three main data sets. (Later to be split to train/test). 

In [282]:
hrce_index = metadata[metadata['cell_type'] == "HRCE"].index.tolist()
vero_index = metadata[metadata['cell_type'] == "VERO"].index.tolist()

meta_hrce = metadata.iloc[hrce_index] ##
meta_vero = metadata.iloc[vero_index]

embed_hrce = embeddings_pc.iloc[hrce_index]
embed_vero = embeddings_pc.iloc[vero_index]

print(np.shape(meta_hrce))
print(np.shape(meta_vero))
print(np.shape(embed_hrce))
print(np.shape(embed_vero))

embed_h_train, embed_h_test, dis_cond_h_train, dis_cond_h_test = train_test_split(
    embed_hrce, meta_hrce['dis_code'], test_size=1/5.0, random_state=0)

print("--------------")
print(np.shape(embed_h_train))
print(np.shape(embed_h_test))
print(np.shape(dis_cond_h_train))
print(np.shape(dis_cond_h_test))

embed_v_train, embed_v_test, dis_cond_v_train, dis_cond_v_test = train_test_split(
    embed_vero, meta_vero['dis_code'], test_size=1/5.0, random_state=0)

print("--------------")
print(np.shape(embed_v_train))
print(np.shape(embed_v_test))
print(np.shape(dis_cond_v_train))
print(np.shape(dis_cond_v_test))


(277656, 12)
(20960, 12)
(277656, 63)
(20960, 63)
--------------
(222124, 63)
(55532, 63)
(222124,)
(55532,)
--------------
(16768, 63)
(4192, 63)
(16768,)
(4192,)


In [283]:
dis_cond_h_train.value_counts()##Do a fast look at the counts. 

0    208622
1      6754
2      6748
Name: dis_code, dtype: int64

In [284]:
dis_cond_h_test.value_counts()

0    52074
2     1732
1     1726
Name: dis_code, dtype: int64

Fitting the first random forest model for HRCE here. Using RF out of the box. Given more time I'd try to maybe tune the hyper params (tree max depth, n_features, etc). Our accuracy for right now is high enough that I feel okay about it. Keep in mind that random forests is a bagging (ensemble) method. Using out-of-bag validation, the model tests against observations not used in bagging. Given more time, we might look at important variables and try to hone down which features are more important than others. If some are really unimportant, we might drop them. Not enough time to delve into that here though. (And our accuracy is already quite good!). 

In [285]:
rf_h = RandomForestClassifier(n_estimators=100, random_state = 0, oob_score = True)
##This is pretty much out of the box random forests. 
## n_estimators = number of trees in the foreset


## Train the model. RF is a bagging method and uses out-of-bag model validation. 
rf_h.fit(embed_h_train, dis_cond_h_train)

dis_h_pred = rf_h.predict(embed_h_test)

rf_h.oob_score_ ##Should be around 95%. With random_state = 0, it is 0.9597747204264285

0.9597747204264285

In [286]:
print("Accuracy:",metrics.accuracy_score(dis_cond_h_test, dis_h_pred))

Accuracy: 0.9601491032197652


Let's now look at a RF model for VERO. Same sort of stuff as before. 

In [287]:
print(dis_cond_v_train.value_counts(), "\n\n", dis_cond_v_test.value_counts())##Do a fast look at the counts. 

0    15717
1      532
2      519
Name: dis_code, dtype: int64 

 0    3963
2     121
1     108
Name: dis_code, dtype: int64


In [288]:
rf_v = RandomForestClassifier(n_estimators=100, random_state = 0, oob_score = True)
##This is pretty much out of the box random forests. 
## n_estimators = number of trees in the foreset


## Train the model. RF is a bagging method and uses out-of-bag model validation. 
rf_v.fit(embed_v_train, dis_cond_v_train)

dis_v_pred = rf_v.predict(embed_v_test)

rf_v.oob_score_ ## With random_state = 0, it is 0.9734613549618321 

0.9734613549618321

In [289]:
print("Accuracy:",metrics.accuracy_score(dis_cond_v_test, dis_v_pred))

Accuracy: 0.9739980916030534


Separated out, HRCE and VERO seem to train up quite nicely. What happens if we try to train a model on all of the data, without regards for cell type?

First we need to create the train and test sets of all the data. 

In [290]:
embed_train, embed_test, dis_cond_train, dis_cond_test = train_test_split(
    embeddings_pc, metadata['dis_code'], test_size=1/5.0, random_state=0)

In [291]:
print(dis_cond_train.value_counts(), "\n\n", dis_cond_test.value_counts())

0    224349
1      7291
2      7252
Name: dis_code, dtype: int64 

 0    56027
2     1868
1     1829
Name: dis_code, dtype: int64


In [292]:
rf_full = RandomForestClassifier(n_estimators=100, random_state = 0, oob_score = True)
##This is pretty much out of the box random forests. 
## n_estimators = number of trees in the foreset


## Train the model. RF is a bagging method and uses out-of-bag model validation. 
rf_full.fit(embed_train, dis_cond_train)

dis_pred = rf_full.predict(embed_test)

rf_full.oob_score_ ## With random_state = 0, it is 0.9608526028498233

0.9608526028498233

In [293]:
print("Accuracy:",metrics.accuracy_score(dis_cond_test, dis_pred))

Accuracy: 0.9612383631370973


The RF model for the full data (both cell types) seems to work pretty well. 

Moving on to scoring the "rescue."

To score the "rescue" of a treatment, I am going to do some semi-supervised learning. The essential concept is that we fit a model based on the mock/UV cells being "non-diseased" and the **untreated** Active-COVID cells as "diseased". From there we predict the probabilities of each treated sample being non-diseased or diseased. All of this is based off of a random forest model.

RFs essentially just do individual tree voting for prediction, meaning that each observation is classified by each tree in the forest and then the class for which the observation gets the most votes (using a 50/50 cutoff) is the predicted class. Given more time to think about the problem, sometimes it may be necessay to use a weighted cutoff (not just 50/50). I'd go back and revisit this if we had a bit more time. 

I am going to setup the data to do binary classification on diseased or non-diseased. The full model on both cell types has quite high accuracy, so I am not going to split out by cell type.

In [294]:
state_index = metadata[metadata['dis_state'] != 0].index.tolist()

meta_state = metadata.iloc[state_index] ##

embed_state = embeddings_pc.iloc[state_index]

embed_unknown = embeddings_pc.loc[~embeddings_pc.index.isin(state_index)]

print(np.shape(meta_state))
print(np.shape(embed_state)) ## (35432, 63)
print(np.shape(embed_unknown)) ## (263184, 63)


embed_state_train, embed_state_test, dis_state_train, dis_state_test = train_test_split(
    embed_state, meta_state['dis_state'], test_size=1/10.0, random_state=0)
##Use a smaller test set so that more of the data goes into training the model. 

print("--------------")
print(np.shape(embed_state_train))
print(np.shape(embed_state_test))
print(np.shape(dis_state_train))
print(np.shape(dis_state_test))

(35432, 12)
(35432, 63)
(263184, 63)
--------------
(31888, 63)
(3544, 63)
(31888,)
(3544,)


Using the embedding -> pca data to look at the Active COVID sites that were treated. 

In [295]:
embed_unknown

,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,...,v_53,v_54,v_55,v_56,v_57,v_58,v_59,v_60,v_61,v_62
0,-5.648822,-9.701595,-4.389027,-3.724540,1.750453,-10.218108,6.698899,-8.002185,2.194457,0.525182,...,1.138334,0.623012,-0.502271,-2.143368,0.012130,-0.953206,1.009449,0.106435,-0.107482,-0.371420
1,-11.236373,-11.383790,-1.669045,-0.880511,-0.002458,-5.740356,6.724205,-5.036800,1.103920,3.161690,...,-2.014080,3.645765,0.137242,-0.486441,-1.513938,1.475253,-0.765367,0.615620,-2.178531,-1.465243
2,-7.570543,-8.762654,2.372159,-3.400681,1.610322,-7.787719,10.060818,-5.434962,-1.070973,2.536703,...,1.119752,0.249184,-0.414499,1.632017,-0.261208,-0.466153,2.193993,-0.877389,0.695840,-1.927933
3,-5.493966,-8.980262,-1.576731,-5.232435,1.818481,-5.916596,2.929959,-8.541632,3.362297,-3.085474,...,0.860601,2.240357,-0.868935,-1.006250,1.326486,-0.609907,-2.981634,1.084425,-0.297733,-2.423678
4,-3.596529,-11.059896,-5.329073,4.466943,6.747674,3.110455,10.896265,-4.314780,-3.210623,6.774098,...,1.607521,-1.724079,-0.893511,-2.345679,0.180880,0.523543,-0.288031,0.841113,0.975545,0.806070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298607,55.044993,-1.114444,-7.325192,-4.311620,-7.560568,2.230893,-0.379839,-2.066260,4.642914,-0.125835,...,1.102712,-0.239736,-0.443787,-0.762842,-0.350501,-0.063724,-0.236836,-1.651119,0.050640,-1.829451
298608,55.203407,-1.147039,-14.526914,-1.812282,-9.791198,-2.949693,2.431965,-8.980934,15.596127,0.762331,...,0.999614,-0.092947,-0.064481,-1.876695,0.627589,-2.287719,1.421402,0.830047,0.083854,1.761823
298609,52.580028,2.366435,-11.814439,-0.256320,-10.249174,-0.021301,7.329787,0.536607,16.507725,6.779081,...,0.601471,0.159341,1.183369,-3.435765,-0.145490,-1.435493,-0.711319,-1.334841,0.391548,0.028311
298610,53.395629,-1.180813,-13.449269,-0.358250,-9.349842,-4.396070,3.709076,-10.718880,14.669375,4.730036,...,-0.478442,1.038667,1.683988,-0.492427,-1.012120,0.551942,-0.842648,1.477071,2.558573,1.388225


Train the RF model on the data with a known disease state. 

In [296]:
rf_ds = RandomForestClassifier(n_estimators=100, random_state = 0, oob_score = True)## ds = disease state
##This is pretty much out of the box random forests. 
## n_estimators = number of trees in the foreset


## Train the model. RF is a bagging method and uses out-of-bag model validation. 
rf_ds.fit(embed_state_train, dis_state_train)

dis_state_pred = rf_ds.predict(embed_state_test)

rf_ds.oob_score_ ## With random_state = 0, it is 0.9721525338685398

0.9721525338685398

In [297]:
print("Accuracy:",metrics.accuracy_score(dis_state_test, dis_state_pred)) ##Accuracy looks really nice. 

Accuracy: 0.9726297968397292


Now we need to run this model on the observations that were treated Active COVID. Get a probability of being non-diseased. That is the score: Probability, as predicted by ``rf_ds``, of being **non-diseased**. 

In [298]:
prob = rf_ds.predict_proba(embed_unknown)

In [299]:
scores = pd.DataFrame(prob[:, 1], columns=["non_dis_prob"], index=embed_unknown.index)
scores.describe() ##Basically what does the scores vector look like?

,non_dis_prob
count,263184.000000
mean,0.141522
std,0.165193
min,0.000000
25%,0.040000
50%,0.080000
75%,0.180000
max,1.000000


In [300]:
scores

,non_dis_prob
0,0.03
1,0.01
2,0.01
3,0.01
4,0.11
...,...
298607,0.07
298608,0.01
298609,0.13
298610,0.00


In [301]:
scores.loc[scores["non_dis_prob"] > 0.5] ## 12371 such observations. 

,non_dis_prob
51,0.56
60,0.51
428,0.51
456,0.52
557,0.86
...,...
298310,0.84
298311,0.68
298572,0.52
298574,0.71


Now that we have the scores, it is just a matter of observing which compounds are used to treat those sites. Find the top 20 "best" compunds. Using > 50% as a cutoff for being non-diseased is just a default. Perhaps with more consideration or background, we could justify changing that cutoff.  

We will also need to consider not just a raw count of times a specific compound "rescued," but we need to look at 

In [302]:
non_dis_index = scores[scores['non_dis_prob'] > 0.5].index.tolist()

In [303]:
treat_compounds = metadata.iloc[non_dis_index]['treatment'] ## Just take the treatment column. 

In [304]:
treat_compounds.value_counts().head(20) ## Need to look at relative values for times treatment was used. Use treat_freq

GS-441524                              138
Remdesivir (GS-5734)                    64
Chloroquine                             58
Hydroxychloroquine Sulfate              47
Quinine                                 36
Polydatin                               36
Indomethacin                            32
Lopinavir                               28
solithromycin                           27
Idelalisib                              27
etazolate                               25
Oseltamivir carboxylate                 25
gepefrine                               24
Arbidol                                 23
MLN2238                                 23
D-Mannitol                              23
Ramosetron                              22
Ulipristal                              22
methylprednisolone-sodium-succinate     21
oxiconazole                             21
Name: treatment, dtype: int64

In [305]:
## Scores weighted by freqency of treatment use. 
weighted_scores = [treat_compounds.value_counts()[a]/(1.0 * treat_freq[a]) for a in treat_compounds.value_counts().index]



The concept here is that we find all compounds that resulted in "rescues." From there, we scale the number of rescues by that treatment by how many times that treatment was used in total. Thus, a treatment with 864 applications, but 20 rescues is not scored as high in the end as a treatment with 144 applications and 18 rescues. 

I am going to get the top 20 many treatments by weighted score. 

In [306]:
top_treatments_rescues = treat_compounds.value_counts()[sorted(range(len(weighted_scores)),
                                                       key=lambda i: weighted_scores[i],
                                                       reverse=True)[:24]]
top_treatments_weighted = pd.DataFrame(sorted(weighted_scores, reverse = True)[:24],
                                       columns=["Weighted Score"],
                                       index=top_treatments_rescues.keys()) 

print(top_treatments_rescues, "\n")


print(top_treatments_weighted)


etazolate                          25
gepefrine                          24
GS-441524                         138
MLN2238                            23
D-Mannitol                         23
Ramosetron                         22
Ulipristal                         22
oxiconazole                        21
Losartan Carboxylic Acid           20
telotristat                        20
gemifloxacin                       20
secnidazole                        20
phenolsulfonphthalein              20
Primidone                          19
Domperidone                        19
4-Biphenylacetic acid              19
Dichlorophen                       19
Mycophenolic acid                  19
nalfurafine                        18
dehydroepiandrosterone-sulfate     18
norethindrone-acetate              18
Tafluprost                         18
Dequalinium                        18
Glipizide                          18
Name: treatment, dtype: int64 

                                Weighted Score
etazolate

The top 20 (plus 4 more that were tied for 20th) treatments are first listed above, along with the number of "rescues" by that treatment. We also include a table of the percentage of a given treatment that resulted in a rescue. 

In [307]:
sorted(weighted_scores, reverse = True)[:25] ##The top 18 outscore the other treatments. We have a 6-way tie after that. 

[0.1736111111111111,
 0.16666666666666666,
 0.1597222222222222,
 0.1597222222222222,
 0.1597222222222222,
 0.1527777777777778,
 0.1527777777777778,
 0.14583333333333334,
 0.1388888888888889,
 0.1388888888888889,
 0.1388888888888889,
 0.1388888888888889,
 0.1388888888888889,
 0.13194444444444445,
 0.13194444444444445,
 0.13194444444444445,
 0.13194444444444445,
 0.13194444444444445,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.11805555555555555]

In [308]:
weighted_scores ##Just doing some spot checking to make sure these values are acctually sensible and correct. 

[0.1597222222222222,
 0.07407407407407407,
 0.08238636363636363,
 0.06676136363636363,
 0.041666666666666664,
 0.041666666666666664,
 0.037037037037037035,
 0.032407407407407406,
 0.03125,
 0.03125,
 0.1736111111111111,
 0.028935185185185185,
 0.16666666666666666,
 0.02662037037037037,
 0.1597222222222222,
 0.1597222222222222,
 0.1527777777777778,
 0.1527777777777778,
 0.024305555555555556,
 0.14583333333333334,
 0.1388888888888889,
 0.1388888888888889,
 0.1388888888888889,
 0.1388888888888889,
 0.1388888888888889,
 0.023148148148148147,
 0.13194444444444445,
 0.13194444444444445,
 0.13194444444444445,
 0.13194444444444445,
 0.13194444444444445,
 0.02199074074074074,
 0.020833333333333332,
 0.125,
 0.125,
 0.125,
 0.020833333333333332,
 0.125,
 0.125,
 0.020833333333333332,
 0.125,
 0.11805555555555555,
 0.11805555555555555,
 0.11805555555555555,
 0.11805555555555555,
 0.11805555555555555,
 0.11805555555555555,
 0.11805555555555555,
 0.11805555555555555,
 0.019675925925925927,
 0.11805

This is the end of the notebook. See the accompanying PDF of other notes. 